In [ ]:
import numpy as np, matplotlib.pyplot as plt

# Estimation of the initial position and velocity for trajectory generation

When generating a new trajectory, we need to use the real world position of the UAV 
when it will start to execute the trajectory. We need the position and the velocity of the UAV in our algorithm.
The expected final state of the UAV is the final state given by the previous trajectory.
However, because of noise, this will not be true.
Instead, we need to consider the state given by the UAV sensors (IMU) and estimate the future state considering the current trajectory.

## Idea 1: Linear acceleration model

Let $t_0$ the beginning of the trajectory,
$t_1$ the beginning of the trajectory generation and 
$t_f = t_0 + T_f$ the time the UAV reach the end of the trajectory.

Usually, we have $t_0 = 0$ and $t_f = T_f$.
Here, we will use the convention $t_1 = 0$ and $t_f = T_f - currentTime$.

A path constraint which is always true is that we constrain the final acceleration to be null.
Then, we can maybe approximate the instant acceleration as an affine function with 
$\vec{a}(t=0)=\vec{a}(0)$ and $\vec{a}(t=t_f)=\vec{0}$.
Because $t_1=0$, we know $\vec{a}(0)$, $\vec{v}(0)$ and $\vec{p}(0)$ as, respectively,
the current acceleration, velocity and position at the time of the trajectory generation.

We model the acceleration as the affine $\vec{a}(t) = \vec{a}(0) - \vec{a}(0) \frac{t}{t_f}$.

By integrating, we deduce the instant velocity and position:

$$\vec{v}(t) = \vec{v}(0) + \vec{a}(0) t - \vec{a}(0)  \frac{t^2}{2 t_f}$$

$$\vec{p}(t) = \vec{p}(0) + \vec{v}(0) t + \vec{a}(t) \frac{t^2}{2} - \vec{a}(0) \frac{t^3}{6 t_f}$$

Finally, we can find the final estimated velocity,

$$\vec{v}(t_f) = \vec{v}(0) + \vec{a}(0) {t_f}{2}$$

and position,

$$\vec{p}(t_f) = \vec{p}(0) + \vec{v}(0) t_f + \vec{a}(0) \frac{t_f^2}{3}$$


### Hypothesis accuracy

We stated the hypothesis that the acceleration could be approximated in all axis by a linear function, between $[t_1, t_f]$,
and equal to $0$ at $t_f$.

To verify it, we plot the instant acceleration during a simple ground truth trajectory.
We use the MAVROS, PX4, Gazebo simulation environment, with the Iris UAV.
The world is `test_zone`, the start point `[0, 0, 1]` and the end point `[5, -7.5, 1]`.
The path is mainly a straight line.
We also set $T_f = 1.5$.
Each new primitive is generated online, 0.7 seconds before reaching the end of the primitive, so $t_1 \approx t_f - 0.7$.
The acceleration data is sampled from the MAVROS topic `/mavros/imu/data`, at the same rate ($\approx 50 Hz$ ?), 
by the python script `plot_acc1.py` in this folder.
The data is stored in the `acc_data1.npy` file, as a numpy object.

In [ ]:
accs = np.load('acc_data1.npy')
imu_rate = 50 # Is that true ?
t = np.linspace(0, accs.shape[0] / imu_rate, accs.shape[0])

# Remove the gravity offset
accs[:, 2] -= 9.807

norms = np.linalg.norm(accs, axis=1)
plt.plot(t, accs[:, 0], label='$a_x$')
plt.plot(t, accs[:, 1], label='$a_y$')
plt.plot(t, accs[:, 2], label='$a_z$')
plt.plot(t, norms, label='$||a||$')
plt.legend()
plt.title('Instant acceleration over time for a straight line diagonal trajectory')
plt.show()